## Описание эксперимента:
Есть мобильное приложение. В этом приложении у пользователей есть возможность покупать игровые предметы за реальные деньги. Чтобы стимулировать пользователей их покупать, приложение периодически предлагает пользователям товары - появляется окошко с рекомендацией купить товар. Отдел машинного обучения предложил улучшение для текущего алгоритма выбора рекомендации. Для проверки улучшений алгоритма был проведен A/B тест. Лог его проведения предоставлен в прикрепленном файле. 

## Метрика: 
средний доход от пользователя за 1 неделю после первого показа ему рекомендации на 10% (после начала A/B теста время первого показа ищется снова)

## Важная информация:
Эксперимент начинается 2023-05-01. Данные есть до 2023-06-01 (но можно завершить раньше, если это позволит оценка длительности)
Вам сказали, что его длительность должна составить 1 месяц.
Все покупки, которые вызваны не влиянием рекомендаций, в этом логе не учитываются

## Описание данных:
- id_product -  идентификатор продукта, который был рекомендован
- is_pay - купил ли пользователь товар
- sum_payment - размер платежа (0, если не купил)
- city - город, в котором находится пользователь
- id_user - пользователь
- timestamp - timestamp события
- date - дата события

## Задачи, которые необходимо решить:
Оценить длительность теста на момент его начала. Сравнить с предложенной. Для оценки необходимо использовать данные с пред экспериментального периода. Посмотреть, есть ли выбросы в данных.
Построить методику расчета целевой метрики. Рассчитать целевую метрику на день окончания теста (рассчитанной в п1) для группы A и B, рассчитать эффект, p_value. Посмотреть, есть ли выбросы в данных.
Рассчитать метрики из п2 по дням и построить их графики.
Принять решение о результате теста - обосновать.

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import scipy.stats as sps

In [2]:
df = pd.read_csv("ab_made_4")

In [3]:
df

,timestamp,id_user,sum_payment,group,city,id_product,is_pay,date
0,1680330573,user_9903,27,NaN,Санкт-Петербург,4.0,1,2023-04-01
1,1680332652,user_6732,0,NaN,Рязань,1.0,0,2023-04-01
2,1680378039,user_4199,0,NaN,Москва,3.0,0,2023-04-01
3,1680337580,user_3606,12,NaN,Санкт-Петербург,7.0,1,2023-04-01
4,1680334389,user_9519,0,NaN,Санкт-Петербург,14.0,0,2023-04-01
...,...,...,...,...,...,...,...,...
56400,1685485266,user_10740,0,A,Санкт-Петербург,14.0,0,2023-05-31
56401,1685481632,user_3589,0,B,Санкт-Петербург,1.0,0,2023-05-31
56402,1685514383,user_10664,13,A,Москва,17.0,1,2023-05-31
56403,1685481325,user_3537,0,B,Ярославь,19.0,0,2023-05-31


In [4]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [5]:
PRE_DATE = '2023-05-01'
EXP_START_DATE = pd.to_datetime('2023-05-01', format='%Y-%m-%d')
EXP_END_DATE = pd.to_datetime('2023-05-31', format='%Y-%m-%d')

In [6]:
df_pre = df[df['date'] < PRE_DATE]
df_a = df[df['group'] == 'A']
df_b = df[df['group'] == 'B']

In [7]:
assert len(df_pre) + len(df_a) + len(df_b) == len(df)

In [8]:
print(f'Len data before experiment: {len(df_pre)}')
print(f'Len data for A group: {len(df_a)}')
print(f'Len data for B group: {len(df_b)}')

Len data before experiment: 25759
Len data for A group: 15389
Len data for B group: 15257


In [9]:
print(f'Number of unique users in A group: {df_a["id_user"].nunique()}')
print(f'Number of unique users in B group: {df_b["id_user"].nunique()}')


Number of unique users in A group: 6000
Number of unique users in B group: 6000


In [10]:
np.intersect1d(df_a['id_user'].values, df_b['id_user'])

array([], dtype=object)

In [11]:
OUTLIER_THRESHOLD = 0.01

In [12]:
def remove_outliers(df, threshold=OUTLIER_THRESHOLD):
    lower_bound = df["sum_payment"].quantile(q=threshold)
    upper_bound = df["sum_payment"].quantile(q=1-threshold)
    emission_df = df[(df["sum_payment"] < lower_bound) | (df["sum_payment"] > upper_bound)]
    
    df.loc[df.index.isin(emission_df.index), "sum_payment"] = \
        df.loc[df.index.isin(emission_df.index), "sum_payment"].apply(lambda x: min(x, upper_bound))
    
    return df

In [13]:
df_pre = remove_outliers(df_pre)

In [14]:
ALPHA = 0.05
BETA = 0.2

In [15]:
def metric(df, ending, exp=False):
    df_one_week = df.groupby('id_user')['timestamp'].min().reset_index(name='min_timestamp')
    df_one_week['max_timestamp'] = df_one_week['min_timestamp'] + (7 * 24 * 60 * 60)
    merged = df.merge(df_one_week, on='id_user')
    merged = merged[(merged['timestamp'] <= merged['max_timestamp']) & (merged['timestamp'] >= merged['min_timestamp'])]
    merged = merged[pd.to_datetime(merged['max_timestamp']).dt.normalize() <= ending]
    
    return merged.groupby('id_user').sum_payment.sum().tolist()

In [16]:
def duration(k, delta_effect, sigma_1, sigma_2, alpha=ALPHA, beta=BETA):
    z = sps.norm.ppf(1 - alpha/2) + sps.norm.ppf(1-beta)
    n = (k+1) * z ** 2 * (sigma_1 ** 2 + sigma_2 **2 / k) / (delta_effect ** 2)
    return n

In [17]:
sigma1 = sigma2 = np.std(metric(df_pre, ending=PRE_DATE))

mean_base = np.mean(metric(df_pre, ending=PRE_DATE))
effect = 0.1 * mean_base

k = df_a['id_user'].nunique() / df_b['id_user'].nunique()

In [18]:
target_n = int(duration(k, effect, sigma1, sigma2, alpha=ALPHA, beta=BETA)/2)

In [19]:
target_n

3609

In [20]:
def find_filter_date(df, target_count):
    start_date = EXP_START_DATE + timedelta(days=7)
    
    while start_date != EXP_END_DATE:
        temp_df = df[df['date'] <= start_date]
        filtered_df = temp_df.groupby('id_user').timestamp.min().reset_index(name='min_timestamp')
        filtered_df['max_timestamp'] = filtered_df['min_timestamp'] + (7 * 24 * 60 * 60)
        filtered_df = filtered_df[pd.to_datetime(filtered_df['max_timestamp']).dt.normalize() <= start_date]
        
        if filtered_df['id_user'].nunique() >= target_count:
            return start_date
        
        start_date = start_date + timedelta(days=1)
    
    return start_date

In [21]:
a_date = find_filter_date(df_a, target_n)
b_date = find_filter_date(df_b, target_n)

if a_date == b_date:
    EXP_END_DATE = a_date
else:
    EXP_END_DATE = max(a_date, b_date)

In [22]:
df_a = df_a[df_a['date'] <= EXP_END_DATE]
df_b = df_b[df_b['date'] <= EXP_END_DATE]

In [23]:
df_a = remove_outliers(df_a)
df_b = remove_outliers(df_b)

In [24]:
def dynamic_metric(df_group_a, df_group_b):
    final_df = pd.DataFrame(
        [],
        columns=["metric_a", "metric_b", "effect", "t", "p_value"],
        index=pd.date_range(EXP_START_DATE + timedelta(days=7), EXP_END_DATE),
    )
    for date in final_df.index:
        tmp_a = metric(df_group_a[df_group_a["date"] <= date], ending=date, exp=True)
        tmp_b = metric(df_group_b[df_group_b["date"] <= date], ending=date, exp=True)
        effect = np.mean(tmp_b) - np.mean(tmp_a)
        results = sps.ttest_ind(tmp_a, tmp_b, equal_var=abs(np.var(tmp_b) - np.var(tmp_a)) <= 0.1)
        t, p_value = results.statistic, results.pvalue
        
        final_df.loc[date] = np.mean(tmp_a), np.mean(tmp_b), effect, t, p_value
    return final_df

In [25]:
results = dynamic_metric(df_a, df_b)
results

,metric_a,metric_b,effect,t,p_value
2023-05-08,7.446429,8.997712,1.551283,-1.770544,0.076988
2023-05-09,7.881508,9.095588,1.21408,-1.523506,0.127924
2023-05-10,8.322009,9.164653,0.842644,-1.123313,0.26151
2023-05-11,8.23652,9.083756,0.847237,-1.229257,0.219161
2023-05-12,8.039832,9.241042,1.20121,-1.880038,0.060263
2023-05-13,8.189608,9.371665,1.182057,-1.989802,0.04674
2023-05-14,8.298887,9.160065,0.861178,-1.551224,0.120977
2023-05-15,8.368569,9.339463,0.970893,-1.860615,0.062903
2023-05-16,8.695013,9.377226,0.682214,-1.344973,0.178732
2023-05-17,8.710079,9.704041,0.993962,-2.041482,0.041278


Т.к. p_value (0.000321) < alpha (0.05), то внедряем новый алгоритм